# 시카고 샌드위치 맛집 분석

## 웹 데이터를 가져오는 Beautiful Soup 익히기

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
page = open("../data/03. test_first.html",'r').read()
print(type(page))
soup = BeautifulSoup(page, 'html.parser')
print(type(soup))
print(soup.prettify())

In [ ]:
list(soup.children)

In [ ]:
html = list(soup.children)[2]
html

In [ ]:
list(html.children)

In [ ]:
body = list(html.children)[3]
body

In [ ]:
soup.body

In [ ]:
list(body.children)

In [ ]:
len(list(body.children))

In [ ]:
soup.find_all('p')

In [ ]:
soup.find('p')

In [ ]:
soup.find_all('p', class_='outer-text')

In [ ]:
soup.find_all(class_="outer-text")

In [ ]:
soup.find_all(id="first")

In [ ]:
soup.find('p')

In [ ]:
list(soup.children)

In [ ]:
soup.head

In [ ]:
soup.head.next_sibling

In [ ]:
soup.head.previous_sibling

In [ ]:
soup.head.next_sibling.next_sibling

In [ ]:
body.p

In [ ]:
body.p.next_sibling.next_sibling

In [ ]:
for each_tag in soup.find_all('p'):
    print(each_tag.get_text())

In [ ]:
body.get_text()

In [ ]:
links = soup.find_all('a')
links

In [ ]:
for each in links:
    href = each['href']
    text = each.string
    print(text + ' -> ' + href)

## 크롬 개발자 도구를 이용해서 원하는 태그 찾기

In [ ]:
from urllib.request import urlopen

In [ ]:
url = "https://finance.naver.com/marketindex/"
page = urlopen(url)

soup = BeautifulSoup(page, "html.parser")

print(soup.prettify())

In [ ]:
soup.find_all('span', 'value')[0].string

## 실전: 시카고 샌드위치 맛집 소개 사이트에 접근하기

In [ ]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

url_base = 'http://www.chicagomag.com'
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub

html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

soup

In [ ]:
print(soup.find_all('div', 'sammy'))

In [ ]:
len(soup.find_all('div', 'sammy'))

In [ ]:
print(soup.find_all('div', 'sammy')[0])

## 접근한 웹 페이지에서 원하는 데이터 추출하고 정리하기

In [ ]:
tmp_one = soup.find_all('div', 'sammy')[0]
type(tmp_one)

In [ ]:
tmp_one.find(class_='sammyRank')

In [ ]:
tmp_one.find(class_='sammyRank').get_text()

In [ ]:
tmp_one.find(class_='sammyListing').get_text()

In [ ]:
tmp_one.find('a')['href']

In [ ]:
import re

tmp_string = tmp_one.find(class_='sammyListing').get_text()

re.split(('\n|\r\n'), tmp_string)

print(re.split(('\n|\r\n'), tmp_string)[0])
print(re.split(('\n|\r\n'), tmp_string)[1])

In [ ]:
from urllib.parse import urljoin

In [ ]:
rank = []
main_menu = []
cafe_name = []
url_add = []

list_soup = soup.find_all('div', 'sammy')

for item in list_soup:
    rank.append(item.find(class_='sammyRank').get_text())
    
    tmp_string = item.find(class_='sammyListing').get_text()

    main_menu.append(re.split(('\n|\r\n'), tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp_string)[1])
    
    url_add.append(urljoin(url_base, item.find('a')['href']))

In [ ]:
from tqdm import tqdm_notebook
import time

rank = []
main_menu = []
cafe_name = []
url_add = []

list_soup = soup.find_all('div', 'sammy')
bar_total = tqdm_notebook(list_soup)

for item in bar_total:
    rank.append(item.find(class_='sammyRank').get_text())
    
    tmp_string = item.find(class_='sammyListing').get_text()

    main_menu.append(re.split(('\n|\r\n'), tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp_string)[1])
    
    url_add.append(urljoin(url_base, item.find('a')['href']))
    
    time.sleep(0.05)

In [ ]:
rank[:5]

In [ ]:
main_menu[:5]

In [ ]:
cafe_name[:5]

In [ ]:
url_add[:5]

In [ ]:
len(rank), len(main_menu), len(cafe_name), len(url_add)

---

In [ ]:
import pandas as pd

data = {'Rank':rank, 'Menu':main_menu, 'Cafe':cafe_name, 'URL':url_add}
df = pd.DataFrame(data)
df.head()

In [ ]:
df = pd.DataFrame(data, columns=['Rank','Cafe','Menu','URL'])
df.head(5)

In [ ]:
df.to_csv('../data/03. best_sandwiches_list_chicago.csv', sep=',', 
          encoding='UTF-8')

## 다수의 웹 페이지에 자동으로 접근해서 원하는 정보 가져오기

In [ ]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

import pandas as pd

In [ ]:
df = pd.read_csv('../data/03. best_sandwiches_list_chicago.csv', index_col=0)
df.head()

In [ ]:
df['URL'][0]

In [ ]:
html = urlopen(df['URL'][0])
soup_tmp = BeautifulSoup(html, "html.parser")
soup_tmp

In [ ]:
print(soup_tmp.find('p', 'addy'))

In [ ]:
price_tmp = soup_tmp.find('p', 'addy').get_text()
price_tmp

In [ ]:
price_tmp.split()

In [ ]:
price_tmp.split()[0]

In [ ]:
price_tmp.split()[0][:-1]

In [ ]:
' '.join(price_tmp.split()[1:-2])

In [ ]:
price = []
address = []

for n in df.index[:3]:
    html = urlopen(df['URL'][n])
    soup_tmp = BeautifulSoup(html, 'lxml')
    
    gettings = soup_tmp.find('p', 'addy').get_text()
    
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))

In [ ]:
price

In [ ]:
address

## Jupyter Notebook에서 상태 진행바를 쉽게 만들어주는 tqdm 모듈
## 상태 진행바까지 적용하고 다시 샌드위치페이지 50개에 접근하기

In [ ]:
from tqdm import tqdm_notebook

price = []
address = []

for n in tqdm_notebook(df.index):
    html = urlopen(df['URL'][n])
    soup_tmp = BeautifulSoup(html, 'lxml')
    
    gettings = soup_tmp.find('p', 'addy').get_text()
    
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2]))

## 50개 웹 페이지에 대한 정보 가져오기

In [ ]:
price

In [ ]:
address

In [ ]:
df.head(10)

In [ ]:
len(price), len(address), len(df)

In [ ]:
df['Price'] = price
df['Address'] = address

df = df.loc[:, ['Rank', 'Cafe', 'Menu', 'Price', 'Address']]
df.set_index('Rank', inplace=True)
df.head()

In [ ]:
df.to_csv('../data/03. best_sandwiches_list_chicago2.csv', sep=',', 
          encoding='UTF-8')

## 맛집 위치를 지도에 표기하기

In [ ]:
import folium
import pandas as pd
import googlemaps
import numpy as np

In [ ]:
df = pd.read_csv('../data/03. best_sandwiches_list_chicago2.csv', index_col=0)
df.head(5)

In [ ]:
gmaps_key = " ******************** " # 2장에서 획득한 자신의 key를 사용합니다.
gmaps = googlemaps.Client(key=gmaps_key)

In [ ]:
lat = []
lng = []

for n in tqdm_notebook(df.index):
    if df['Address'][n] != 'Multiple':
        target_name = df['Address'][n]+', '+'Chicago'
        gmaps_output = gmaps.geocode(target_name)
        location_output = gmaps_output[0].get('geometry')
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lng'])
        
    else:
        lat.append(np.nan)
        lng.append(np.nan)

In [ ]:
len(lat), len(lng)

In [ ]:
df['lat'] = lat
df['lng'] = lng
df.head()

In [ ]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], 
                                      zoom_start=11)
folium.Marker([df['lat'].mean(), df['lng'].mean()], 
                                      popup='center').add_to(mapping)
mapping

In [ ]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], 
                     zoom_start=11)

for n in df.index:
    if df['Address'][n] != 'Multiple':
        folium.Marker([df['lat'][n], df['lng'][n]], 
                                      popup=df['Cafe'][n]).add_to(mapping)

mapping